# AI-Powered Chatbot for Mental Health Support

## Problem Statement
Mental health support is often inaccessible due to stigma, high costs, and a shortage of mental health professionals.

## Solution
Introducing a  chatbot using LLaMA that provides empathetic responses, suggests coping mechanisms, and directs users to appropriate resources while maintaining user privacy.

## Impact Sector
Healthcare. Provides accessible mental health support, reducing barriers to care and promoting well-being.

## Work Breakdown Structure (WBS)
1. **Requirements Analysis**
2. **Data Collection and Preparation**
   - For data we had use the mental health book
3. **AI Model Development**
   - LLaMA integration
4. **User Interaction Design**
   - Chat interface
5. **Privacy and Security Protocols**
6. **Testing and User Feedback**
7. **Deployment and Monitoring**



## Importing Required Libraries

In [ ]:
import streamlit as st
from streamlit_chat import message
from langchain.chains import ConversationalRetrievalChain
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import CTransformers
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory

In [2]:
# Load the PDF files from the path
loader = DirectoryLoader('data/', glob="*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [3]:
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
text_chunks = text_splitter.split_documents(documents)

In [ ]:
# Create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': "cpu"})

In [5]:
# Vectorstore
vector_store = FAISS.from_documents(text_chunks, embeddings)

In [6]:
# Create LLM
llm = CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin", model_type="llama", config={'max_new_tokens': 128, 'temperature': 0.01})

In [ ]:
# Memory (make sure to follow migration guide if further updates are needed)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [8]:
# Conversational Retrieval Chain
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type='stuff',
    retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
    memory=memory
)

In [ ]:
# Streamlit UI Setup
st.title("Mental HealthCare ChatBot 🧑🏽‍⚕️")
def conversation_chat(query):
    result = chain({"question": query, "chat_history": st.session_state['history']})
    st.session_state['history'].append((query, result["answer"]))
    return result["answer"]

In [12]:
def initialize_session_state():
    if 'history' not in st.session_state:
        st.session_state['history'] = []

    if 'generated' not in st.session_state:
        st.session_state['generated'] = ["Hello! Ask me anything about 🤗"]

    if 'past' not in st.session_state:
        st.session_state['past'] = ["Hey! 👋"]

In [13]:
def display_chat_history():
    reply_container = st.container()
    container = st.container()

    with container:
        with st.form(key='my_form', clear_on_submit=True):
            user_input = st.text_input("Question:", placeholder="Ask about your Mental Health", key='input')
            submit_button = st.form_submit_button(label='Send')

        if submit_button and user_input:
            output = conversation_chat(user_input)

            st.session_state['past'].append(user_input)
            st.session_state['generated'].append(output)

    if st.session_state['generated']:
        with reply_container:
            for i in range(len(st.session_state['generated'])):
                message(st.session_state["past"][i], is_user=True, key=str(i) + '_user', avatar_style="thumbs")
                message(st.session_state["generated"][i], key=str(i), avatar_style="fun-emoji")

In [ ]:
# Initialize session state
initialize_session_state()
# Display chat history
display_chat_history()